In [1]:
import numpy as np
import pickle
from sklearn.linear_model import LinearRegression
from scipy.spatial import distance_matrix
from pysal.lib import weights
from esda import Moran

/usr/local/lib/python3.8/dist-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.4-CAPI-1.17.4) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [11]:
path = '/root/code/paper_public/data/10ms'

with open(f"{path}/date_all.pkl","rb") as f:
    date_all = pickle.load(f)
# with open("/root/code/paper_public/data/xy_all.pkl","rb") as f:
#     xy_all = pickle.load(f)
# with open("/root/code/paper_public/data/varspec_all.pkl","rb") as f:
#     varspec_all = pickle.load(f)
# with open("/root/code/paper_public/data/varspec_shuffle_all.pkl","rb") as f:
#     varspec_shuffle_all = pickle.load(f)
# with open("/root/code/paper_public/data/U_all.pkl","rb") as f:
#     U_all = pickle.load(f)
# with open("/root/code/paper_public/data/U_shuffle_all.pkl","rb") as f:
#     U_shuffle_all = pickle.load(f)
with open(f"{path}/size_all.pkl","rb") as f:
    size_all = pickle.load(f)
with open(f"{path}/duration_all.pkl","rb") as f:
    duration_all = pickle.load(f)
with open(f"{path}/size_time_all.pkl","rb") as f:
    size_time_all = pickle.load(f)
with open(f"{path}/duration_time_all.pkl","rb") as f:
    duration_time_all = pickle.load(f)
with open(f"{path}/burstiness_all.pkl","rb") as f:
    burstiness_all = pickle.load(f)

In [5]:
## linear fitting in log-log scale (power fitting)
def get_linear(x, y, yllim): # llim: lower limit for cutoff
    n = len(x)

    x_fit = np.zeros(n)
    y_fit = np.zeros(n)
    count = 0

    for i in range(n):
        if y[i] > yllim:
            x_fit[count] = np.log10(x[i]+1)
            y_fit[count] = np.log10(y[i])
            count += 1
                
    model = LinearRegression()
    model.fit(x_fit[:count].reshape(-1, 1), y_fit[:count])
    a = model.coef_
    b = model.intercept_
    error = np.average((a*x_fit[:count]+b - y_fit[:count])**2) # mean squared error
    return a, b, error
    
## generate histogram
def get_hist(size, duration, size_max, dura_max, dura_dt):
    size_hist = np.zeros(size_max)
    for s in range(size_max):
        size_hist[s] = np.sum(size == s)
        if np.sum(size_hist[:s]) == len(size):
            break
    size_hist /= np.sum(size_hist)
    
    duration_hist = np.zeros(int(dura_max/dura_dt))
    for i_d in range(int(dura_max/dura_dt)):
        d = i_d*dura_dt
        sum_temp = np.sum(duration_hist[:i_d])
        duration_hist[i_d] = np.sum(duration < d+dura_dt) - sum_temp
        if sum_temp == len(duration):
            break
    duration_hist /= np.sum(duration_hist)
    
    return size_hist, duration_hist

generate histogram of cascade size and distribution (correspond to Fig.3a, b, c, e, f, g)

In [9]:
size_hist_all = {} # histogram of cascade size
duration_hist_all = {} # histogram of cascade duration
size_max = 50000 # max. number of firing in one cascade
dura_max = 10000 # max. duration [ms]
dura_dt = 1 # [ms]

for dish, date_list in date_all.items():
    size_hist_dish = []
    duration_hist_dish = []
    
    for d in range(len(date_list)):
        size = np.array(size_time_all[dish][d])
        duration = np.array(duration_time_all[dish][d])*1000 # [s] to [ms]
        
        size_hist, duration_hist = get_hist(size, duration, size_max, dura_max, dura_dt)
        size_hist_dish.append(size_hist)
        duration_hist_dish.append(duration_hist)
        
    size_hist_all[dish] = size_hist_dish
    duration_hist_all[dish] = duration_hist_dish

In [10]:
path = '/root/code/paper_public/result/10ms'

with open(f"{path}/size_time_hist_all.pkl","wb") as f:
    pickle.dump(size_hist_all, f)
with open(f"{path}/duration_time_hist_all.pkl","wb") as f:
    pickle.dump(duration_hist_all, f)

Calculate exponent, intercept, and fitting error for power-law dimensionality and neuronal avelanche (correspond to Fig.1d, e, f, Fig3.b, c, g)

In [22]:
Dim_abe_all = {} # {a: exponent, b: intercept, e: error} of power-law fitting to Variance spectre
Dim_abe_shuffle_all = {} # {a: exponent, b: intercept, e: error} of power-law fitting to Variance spectre if shuffled data
x_log = np.array([2,4,8,16,32,64,128,256])-1
for dish, date_list in date_all.items():
    if dish in varspec_all:
        Dim_abe_dish = []
        Dim_abe_shuffle_dish = []
        for d in range(len(date_list)):
            a, b, e = get_linear(x_log, varspec_all[dish][d][x_log], 0)
            Dim_abe_dish.append([a, b, e])
            
            abe_temp = []
            N_shuffle = len(varspec_shuffle_all[dish][d][:,0])
            for i_shuffle in range(N_shuffle):
                a, b, e = get_linear(x_log, varspec_shuffle_all[dish][d][i_shuffle,x_log], 0)
                abe_temp.append([a, b, e])
            Dim_abe_shuffle_dish.append(abe_temp)
    Dim_abe_all[dish] = Dim_abe_dish
    Dim_abe_shuffle_all[dish] = Dim_abe_shuffle_dish
    

NA_abe_all = {} # {a: exponent, b: intercept, e: error} of power-law fitting to Cascade size distribution
for dish, date_list in date_all.items():
    if dish in size_hist_all:
        NA_abe_dish = []
        for d in range(len(date_list)):
            # 対数スケール
            size_hist = size_hist_all[dish][d]
            size_hist_interpolate = np.zeros(len(size_hist))
            for j in np.arange(len(size_hist)-2,-1,-1):
                if size_hist[j] == 0:
                    size_hist_interpolate[j] = size_hist_interpolate[j+1]
                else:
                    size_hist_interpolate[j] = size_hist[j]
                    
            x_log = [int(x) for x in 2**np.arange(0,np.log2(np.count_nonzero(size_hist_interpolate)))]
            a, b, e = get_linear(x_log, size_hist_interpolate[x_log], 0)
            NA_abe_dish.append([a, b, e])
            
    NA_abe_all[dish] = NA_abe_dish

In [23]:
with open("/root/code/paper_public/result/Dim_abe_all.pkl","wb") as f:
    pickle.dump(Dim_abe_all, f)
with open("/root/code/paper_public/result/Dim_abe_shuffle_all.pkl","wb") as f:
    pickle.dump(Dim_abe_shuffle_all, f)
with open("/root/code/paper_public/result/NA_abe_all.pkl","wb") as f:
    pickle.dump(NA_abe_all, f)

Calculate locality and locality slope (correspond to Fig.2c, d)

In [7]:
locality_all = {}
locality_shuffle_all = {}


r = 200 # threshold distance [μm]
thresh = 0.0001 # threshold local moran I 
N = 1024

for dish, date_list in date_all.items():
    if dish in U_all:
        locality_dish = []
        locality_shuffle_dish = []
        for d in range(len(date_list)):
            xy = xy_all[dish][d]
            distance = distance_matrix(xy, xy)
            w_r = np.zeros((N,N))
            w_r[distance < r] = 1
            np.fill_diagonal(w_r,0)
            w_r = w_r / (np.sum(w_r, axis=1, keepdims=True))
            
            U = U_all[dish][d]
            U_norm = U - np.mean(U, axis=0, keepdims=True)
            local_moran_r = U_norm*(w_r@U_norm)
            locality_dish.append(np.sum(local_moran_r > thresh,axis=0))
            
            locality_temp = []
            N_shuffle = len(U_shuffle_all[dish][d][:,0,0])
            for i_shuffle in range(N_shuffle):
                U = U_shuffle_all[dish][d][i_shuffle,:,:]
                U_norm = U - np.mean(U, axis=0, keepdims=True)
                local_moran_r = U_norm*(w_r@U_norm)
                locality_temp.append(np.sum(local_moran_r > thresh,axis=0))
            
            locality_shuffle_dish.append(locality_temp)
        
        locality_all[dish] = locality_dish
        locality_shuffle_all[dish] = locality_shuffle_dish

<ipython-input-7-bd8edb29ebee>:19: RuntimeWarning: invalid value encountered in divide
  w_r = w_r / (np.sum(w_r, axis=1, keepdims=True))


In [40]:
locality_slope_all = {}
locality_slope_shuffle_all = {}

N = 1024
Nshuffle = 10
window = 10
x_fit = 2**np.arange(3,11)
model = LinearRegression()

for dish, date_list in date_all.items():
    if dish in locality_all:
        locality_slope_dish = []
        locality_slope_shuffle_dish = []
        for d in range(len(date_list)):
            locality = locality_all[dish][d]
            locality_shuffle = np.array(locality_shuffle_all[dish][d])
            
            locality_mvavg = np.zeros(len(locality))
            locality_shuffle_mvavg = np.zeros(np.shape(locality_shuffle))
            mvavg_counter = np.zeros(len(locality))
            for w in range(window):
                locality_mvavg[:N-w] += locality[w:]
                locality_shuffle_mvavg[:,:N-w] += locality_shuffle[:,w:]
                mvavg_counter[:N-w] += 1
            locality_mvavg /= mvavg_counter
            locality_shuffle_mvavg /= mvavg_counter
            
            model.fit(np.log10(x_fit).reshape(-1, 1), locality_mvavg[x_fit-1])
            a,b = model.coef_, model.intercept_
            locality_slope_dish.append(a)
            
            slope_temp = []
            for i_shuffle in range(N_shuffle):
                model.fit(np.log10(x_fit).reshape(-1, 1), locality_shuffle_mvavg[i_shuffle][x_fit-1])
                a,b = model.coef_, model.intercept_
                slope_temp.append(a)
            locality_slope_shuffle_dish.append(slope_temp)
            
        locality_slope_all[dish] = locality_slope_dish
        locality_slope_shuffle_all[dish] = locality_slope_shuffle_dish

In [46]:
with open("/root/code/paper_public/result/locality_all.pkl","wb") as f:
    pickle.dump(locality_all, f)
with open("/root/code/paper_public/result/locality_shuffle_all.pkl","wb") as f:
    pickle.dump(locality_shuffle_all, f)
with open("/root/code/paper_public/result/locality_slope_all.pkl","wb") as f:
    pickle.dump(locality_slope_all, f)
with open("/root/code/paper_public/result/locality_slope_shuffle_all.pkl","wb") as f:
    pickle.dump(locality_slope_shuffle_all, f)